In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pickle

# Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')

# Load dữ liệu
data = pd.read_csv('./train-data/FINANCIAL_DATA.csv')
# Hàm tạo dataset dạng sliding window với target đã có sẵn (không dùng scaler)
def create_multifeature_dataset(df):
    X = df[columns].values
    Y = df['target'].values
    return X, Y


time_step = 1
X, y = create_multifeature_dataset(data)

X = X.reshape((X.shape[0], 1, X.shape[1]))

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình LSTM cho bài toán phân loại nhị phân
model = Sequential([
    Input(shape=(time_step, len(columns))),
    LSTM(300, return_sequences=True),
    LSTM(300, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Dùng hàm kích hoạt sigmoid cho đầu ra nhị phân
])

# Compile model với loss cho phân loại nhị phân
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision()])

# Định nghĩa callback dừng sớm nếu val_accuracy không cải thiện
early_stopping = EarlyStopping(monitor='val_precision', patience=5, restore_best_weights=True)

# Huấn luyện mô hình
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          batch_size=64, verbose=1, epochs=1000, callbacks=[early_stopping])

# Lưu model (không lưu scaler vì không sử dụng)
model.save('./model/predict_model_XAUUSD_v2.h5')


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')

# Hàm tạo dataset dạng sliding window với target đã có sẵn (không dùng scaler)
def create_multifeature_dataset(df):
    X = df[columns].values
    Y = df['target'].values
    return X, Y

chunksize = 200000

# Generator đọc dữ liệu theo từng chunk
def data_generator(file_path, chunksize=300000):
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        X_chunk, y_chunk = create_multifeature_dataset(chunk)
        # Reshape dữ liệu cho LSTM: (samples, time_step, features)
        X_chunk = X_chunk.reshape((X_chunk.shape[0], 1, X_chunk.shape[1]))
        yield X_chunk, y_chunk

# Xây dựng mô hình LSTM cho bài toán phân loại nhị phân
time_step = 1
model = Sequential([
    Input(shape=(time_step, len(columns))),
    LSTM(400, return_sequences=True),
    LSTM(200, return_sequences=False),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile model với loss cho phân loại nhị phân
model.compile(optimizer=Adam(learning_rate=0.0005), 
            loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision()]
)

# Định nghĩa callback dừng sớm nếu val_precision không cải thiện


file_path = './FOREX.csv'
# Huấn luyện mô hình theo từng chunk từ generator với validation
for X_chunk, y_chunk in data_generator(file_path, chunksize=chunksize):
    # Tách dữ liệu của mỗi chunk thành train và validation
    early_stopping = EarlyStopping(monitor='val_precision', patience=8, restore_best_weights=True)
    X_train, X_val, y_train, y_val = train_test_split(X_chunk, y_chunk, test_size=0.2, random_state=42)
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=64, epochs=50, callbacks=[early_stopping], verbose=1)

# Sau khi training xong, lưu model cuối cùng
model.save('./model/predict_model_XAUUSD_v1.h5')


In [2]:
import tensorflow as tf

# Đường dẫn đến file CSV
file_path = './train-data/FOREX.csv'
batch_size = 64

# 1. Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')
target_column = 'target'
all_columns = columns + [target_column]

# 2. Tạo dataset streaming từ CSV (không load toàn bộ vào bộ nhớ)
raw_dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=batch_size,
    header=True,
    shuffle=True,
    num_epochs=1,
).ignore_errors()

# 3. Hàm xử lý mỗi batch: sắp xếp lại thứ tự các cột và reshape cho LSTM
def parse_csv(batch):
    # Lấy các cột features theo thứ tự mong muốn
    features = [batch[col] for col in columns]  # mỗi tensor có shape (batch_size,)
    X = tf.stack(features, axis=1)              # shape: (batch_size, num_features)
    X = tf.expand_dims(X, axis=1)                 # reshape thành (batch_size, time_steps=1, num_features)
    y = batch[target_column]
    return X, y

dataset = raw_dataset.map(parse_csv)

# 4. Unbatch và enumerate để xử lý từng mẫu riêng lẻ, từ đó chia theo tỷ lệ cố định
dataset = dataset.unbatch()       # Tách ra thành từng mẫu đơn
dataset = dataset.enumerate()     # Gán chỉ số cho mỗi mẫu: (index, (X, y))

# 5. Chia dữ liệu theo modulo (ví dụ: 80% train, 20% validation)
def is_train(idx, data):
    # Với mỗi 5 mẫu, 4 mẫu đầu cho train (idx mod 5 < 4)
    return tf.math.mod(idx, 5) < 4

def is_val(idx, data):
    return tf.math.mod(idx, 5) == 4

train_dataset = dataset.filter(is_train).map(lambda idx, data: data)
val_dataset   = dataset.filter(is_val).map(lambda idx, data: data)

# 6. Re-batch lại dữ liệu
train_dataset = train_dataset.batch(batch_size)
val_dataset   = val_dataset.batch(batch_size)

# (Tùy chọn) Nếu cần lặp lại dataset vô hạn:
train_dataset = train_dataset.repeat()
val_dataset   = val_dataset.repeat()

# 7. Tính số bước (steps) dựa trên số dòng trong file CSV
# Đếm số dòng trong file (trừ header)
with open(file_path, 'r') as f:
    total_samples = sum(1 for line in f) - 1

# Vì ta chia theo tỉ lệ 80/20, ta tính số steps cho train và validation
total_steps = total_samples // batch_size
train_steps = int(0.8 * total_steps)
validation_steps = int(0.2 * total_steps)

# 8. Xây dựng mô hình LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(1, len(columns))),
    tf.keras.layers.LSTM(400, return_sequences=True),
    tf.keras.layers.LSTM(200, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision()])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Sử dụng 'val_loss' để đảm bảo metric có giá trị ngay từ epoch đầu
    patience=8,
    restore_best_weights=True
)

# 9. Huấn luyện mô hình
model.fit(train_dataset,
          validation_data=val_dataset,
          epochs=50,
          steps_per_epoch=train_steps,
          validation_steps=validation_steps,
          callbacks=[early_stopping],
          verbose=1)

# 10. Sau khi training, lưu model
model.save('./predict_model_XAUUSD_v2.h5')


Epoch 1/50
 14292/207442 ━━━━━━━━━━━━━━━━━━━━ 19:15 6ms/step - loss: 0.6739 - precision: 0.5795

KeyboardInterrupt: 

In [ ]:
import dask.dataframe as dd

# Đọc file CSV lớn; Dask tự động chia thành nhiều partition
df = dd.read_csv('./train-data/FOREX.csv')

# Xáo trộn toàn bộ dữ liệu (lấy mẫu ngẫu nhiên với frac=1)
df_shuffled = df.sample(frac=1, random_state=42)

# Gom tất cả các partition thành 1 partition
df_single = df_shuffled.repartition(npartitions=1)

# Ghi ra file CSV duy nhất (tham số single_file=True yêu cầu Dask phiên bản hỗ trợ)
df_single.to_csv('FOREX.csv', index=False, single_file=True)


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import math

# Định nghĩa lớp TransformerBlock
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Xây dựng danh sách các cột features với các lag
lags = range(0, 15)
columns = []
for col in ['body', 'shadow_top', 'shadow_bottom']:
    for lag in lags:
        columns.append(f'{col}_{lag}')

# Tham số đọc CSV và huấn luyện
csv_file = './train-data/FINANCIAL_DATA.csv'
chunksize = 50000   # Số dòng mỗi chunk khi đọc file
batch_size = 64     # Số mẫu mỗi batch huấn luyện
time_step = 1       # Lưu ý: với Transformer bạn có thể cân nhắc dùng chuỗi dài hơn nếu có thể

# Generator đọc CSV theo từng phần (chunk)
def data_generator(file_path, chunksize):
    # Đọc CSV theo từng chunk để tiết kiệm bộ nhớ
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        # Lấy các cột features và cột target
        X_chunk = chunk[columns].values
        y_chunk = chunk['target'].values
        # Reshape X_chunk thành dạng (số mẫu, time_step, số feature)
        X_chunk = X_chunk.reshape((X_chunk.shape[0], time_step, X_chunk.shape[1]))
        # Yield từng mẫu một để tf.data có thể batch sau này
        for i in range(len(X_chunk)):
            yield X_chunk[i], y_chunk[i]

# Hàm đếm số dòng trong file CSV (loại header)
def count_csv_rows(file_path):
    with open(file_path, 'r') as f:
        num_lines = sum(1 for line in f) - 1
    return num_lines

num_samples = count_csv_rows(csv_file)
# Chia 80% train và 20% validation
num_train = int(num_samples * 0.8)
num_val = num_samples - num_train

# Tạo tf.data.Dataset từ generator
dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(csv_file, chunksize),
    output_types=(tf.float32, tf.float32),
    output_shapes=((time_step, len(columns)), ())
)

# Tách dataset thành train và validation dựa trên số dòng đã tính
train_dataset = dataset.take(num_train)
val_dataset = dataset.skip(num_train)

# Batch và prefetch để tăng hiệu năng đọc dữ liệu
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Tham số cho mô hình Transformer
embed_dim = 64       # Kích thước embedding
num_heads = 4        # Số heads của MultiHeadAttention
ff_dim = 128         # Số nút trong feed-forward network
dropout_rate = 0.2
num_transformer_blocks = 2  # Số lớp transformer

# Xây dựng mô hình Transformer bằng Functional API
inputs = Input(shape=(time_step, len(columns)))
x = Dense(embed_dim)(inputs)
for _ in range(num_transformer_blocks):
    x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(dropout_rate)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile model với loss và metric cho phân loại nhị phân
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision()])

# Callback dừng sớm nếu val_precision không cải thiện sau 5 epochs
early_stopping = EarlyStopping(monitor='val_precision', patience=5, restore_best_weights=True)

# Huấn luyện mô hình sử dụng dataset được tạo từ generator
model.fit(
    train_dataset,
    epochs=1000,
    validation_data=val_dataset,
    callbacks=[early_stopping],
    verbose=1
)

# Lưu model đã huấn luyện
model.save('./model/predict_model_XAUUSD_transformer_v2.h5')


FileNotFoundError: [Errno 2] No such file or directory: './train-data/FINANCIAL_DATA.csv'